In [1]:
from  pathlib import Path
from PIL import Image
import cv2
import numpy as np
from multiprocessing import Pool, cpu_count
from tqdm import tqdm 

In [2]:
cutouts = [ x for x in Path("/home/weedsci/matt/SemiF-AnnotationPipeline/data/semifield-cutouts").rglob("*mask.png")]
len(cutouts)


117009

In [3]:
def convert(imgpath):
    img = cv2.imread(str(imgpath))
    if img is not None:
        if img.max() == 255:
            img = np.where(img!= 0, 1, 0)
            cv2.imwrite(str(imgpath), img)

cpus = cpu_count() -3
with Pool(cpus) as p:
    p.map(convert, cutouts)
    p.close()
    p.join()



    
